In [ ]:
import sagemaker
import json

# default configuraton dict with some default values

conf_dict = {}
conf_dict["ROLE"] = sagemaker.get_execution_role()

BUCKET_NAME="PUT_BUCKET_NAME_HERE"

# overall configuration
conf_dict["REGION"] = "eu-central-1"
conf_dict["MODEL_PACKAGE_NAME"] = "XgBoostWorkshopClassification-modelsPackageGroup"
conf_dict["PIPELINE_NAME"] = "WORKSHOPS-AutoGluon"
conf_dict["INPUT_PATH"] = f"s3://{BUCKET_NAME}/Skyserver_df.csv"
conf_dict["ARTIFACTS_SAVE_PATH"] = f"s3://{BUCKET_NAME}/outputs"
conf_dict["MODEL_SAVE_PATH"] = f'{conf_dict["ARTIFACTS_SAVE_PATH"]}/run_1'

# preprocessing step configuration
conf_dict["PREPROCESSING_STEP_CONFIG"] = {}
conf_dict["PREPROCESSING_STEP_CONFIG"]["instance_type"] = "ml.m5.large"
conf_dict["PREPROCESSING_STEP_CONFIG"]["instance_count"] = 1
conf_dict["PREPROCESSING_STEP_CONFIG"]["framework_version"] = "1.0-1"

# model training step configuration
conf_dict["MODEL_TRAINING_STEP_CONFIG"] = {}
conf_dict["MODEL_TRAINING_STEP_CONFIG"]["instance_type"] = "ml.p3.2xlarge"
conf_dict["MODEL_TRAINING_STEP_CONFIG"]["instance_count"] = 1
conf_dict["MODEL_TRAINING_STEP_CONFIG"]["python_version"] = "py3"

# model evaluation step configuration
conf_dict["MODEL_EVALUATION_STEP_CONFIG"] = {}
conf_dict["MODEL_EVALUATION_STEP_CONFIG"]["instance_type"] = "ml.m5.large"
conf_dict["MODEL_EVALUATION_STEP_CONFIG"]["instance_count"] = 1

conf_dict["CREATE_MODEL_CONFIG"] = {}
conf_dict["CREATE_MODEL_CONFIG"]["instance_type"] = "ml.m5.large"

json_object = json.dumps(conf_dict, indent=4)

# Save configuration into .json file
with open("config.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
from pprint import pprint
import json
import sagemaker

from scripts.pipeline import get_pipeline

role = sagemaker.get_execution_role()

# Opening saved configiration file
with open("config.json", "r") as openfile:
    # Reading from json file
    config_dict = json.load(openfile)

"""
Running pipeline creation

- run this cell

"""
pipeline = get_pipeline(
    config_dict=config_dict,
)

### 5. Pipeline execution

In [ ]:
# pipeline execution

definition = json.loads(pipeline.definition())

with open("data.json", "w") as f:
    json.dump(definition, f)

pipeline.upsert(role_arn=role)
execution = pipeline.start()

### 6. Run this cell if you want to wait for the pipeline to be created

In [ ]:
execution.wait()